In [1]:
%tensorflow_version 1.x

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777) # reproducibility

In [3]:
sample = " if you want you"
idx2char = list(set(sample)) # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)} # char -> index

In [4]:
# hyper parameters
dic_size = len(char2idx) # RNN input size (one hot size)
hidden_size = len(char2idx) # RNN output size
num_classes = len(char2idx) # final output size (RNN or softmax, etc.)
batch_size = 1 # one sample data, one batch
sequence_length = len(sample) - 1 # number of lstm rollings (unit #)
learning_rate = 0.1

In [5]:
sample_idx = [char2idx[c] for c in sample] # char to index
x_data = [sample_idx[:-1]] # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1::]] # Y label sample (1 ~ n) hello: ello

In [6]:
X = tf.placeholder(tf.int32, [None, sequence_length]) # X data
Y = tf.placeholder(tf.int32, [None, sequence_length]) # Y label

In [7]:
x_one_hot = tf.one_hot(X, num_classes) # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

In [8]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state = initial_state, dtype = np.float32
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn = None)

Instructions for updating:
Please use `layer.__call__` method instead.


In [10]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [11]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits = outputs, targets = Y, weights = weights
)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [12]:
prediction = tf.argmax(outputs, axis = 2)

In [13]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for i in range(50):
    l, _ = sess.run([loss, train], feed_dict = {X: x_data, Y: y_data})
    result = sess.run(prediction, feed_dict = {X: x_data})

    # print char using dic
    result_str = [idx2char[c] for c in np.squeeze(result)]
    print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.3035893 Prediction: yu yyuuyuuuuyou
1 loss: 2.1593056 Prediction: y  yyuuyuuuyyou
2 loss: 1.988271 Prediction: y  youuyanuyyou
3 loss: 1.75305 Prediction: y  you yantyyou
4 loss: 1.443023 Prediction: y  you  ant you
5 loss: 1.1391722 Prediction: y  you  ant you
6 loss: 0.8601469 Prediction: y  you want you
7 loss: 0.62253034 Prediction: yf you want you
8 loss: 0.45680514 Prediction: yf you want you
9 loss: 0.33173797 Prediction: yf you want you
10 loss: 0.24530602 Prediction: yf you want you
11 loss: 0.18715096 Prediction: yf you want you
12 loss: 0.14433397 Prediction: if you want you
13 loss: 0.11137915 Prediction: if you want you
14 loss: 0.08526446 Prediction: if you want you
15 loss: 0.06394668 Prediction: if you want you
16 loss: 0.046565738 Prediction: if you want you
17 loss: 0.03301714 Prediction: if you want you
18 loss: 0.023135034 Prediction: if you want you
19 loss: 0.01632738 Prediction: if you want you
20 loss: 0.011789935 Prediction: if you want you
21 loss: 0